In [1]:
from google import genai
from google.genai import types
import torch
from transformers import pipeline
import scipy
import json
import os
from pathlib import Path

In [4]:
pip list

Note: you may need to restart the kernel to use updated packages.


D:\ai_program\submissions\image_narration_tool\.venv\Scripts\python.exe: No module named pip


In [2]:
BASE_DIR = Path(os.getcwd())

PROJECT_ROOT = BASE_DIR.parent

with open(f'{PROJECT_ROOT}/images/image_2.jpeg', 'rb') as f:
  image_bytes = f.read()

with open('api_key.txt','r') as f:
    api_key_value = f.read()

In [3]:
client = genai.Client(api_key=api_key_value)

In [4]:
client = genai.Client(api_key=api_key_value)

context = [
    types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
    ),
    types.Part.from_text(text="You are a helpful AI Assistant. Given an image perform object detection and provide a text output which contains the information about the labels detected and their counts.")
]

model_response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=context,
)
text_from_model = model_response.text
print(text_from_model)

The image shows a busy multi-lane highway at night, filled with traffic. My analysis has detected the following objects:

*   **car:** 64
*   **person:** 5
*   **bus:** 2
*   **truck:** 1
*   **motorcycle:** 1

The highway is packed with numerous **cars**, from large SUVs to smaller sedans, stretching far into the distance. A few **buses** are interspersed among the cars, including a prominent red and white one. On the far left, a large white **truck** is partially visible. Amidst the heavy traffic, a single **motorcycle** with a rider in red is weaving through. Along the sides of the road and on walkways, a few **people** can be seen.


In [5]:
labels=None

sentence = "I see "

if (isinstance(text_from_model, str)):
    sentence += text_from_model
    
else:
    formatted_model_response = json.loads(text_from_model)
    labels = [x['label'] for x in text_from_model]
    objects = {}
    for label in labels:
        if objects.get(label):
            objects[label] += 1
        else:
            objects[label] = 1

    print(labels)

    for object in objects.keys():
        sentence += f"{objects[object]} {object}" + 's ' if objects[object] >1 else ''
print(sentence)

I see The image shows a busy multi-lane highway at night, filled with traffic. My analysis has detected the following objects:

*   **car:** 64
*   **person:** 5
*   **bus:** 2
*   **truck:** 1
*   **motorcycle:** 1

The highway is packed with numerous **cars**, from large SUVs to smaller sedans, stretching far into the distance. A few **buses** are interspersed among the cars, including a prominent red and white one. On the far left, a large white **truck** is partially visible. Amidst the heavy traffic, a single **motorcycle** with a rider in red is weaving through. Along the sides of the road and on walkways, a few **people** can be seen.


In [6]:
speech_audio = pipeline(
    task="text-to-audio",
    model="suno/bark-small"
)

gen_audio = speech_audio(sentence)

scipy.io.wavfile.write("traffic_sentence_gemini.wav", rate=gen_audio["sampling_rate"], data=gen_audio["audio"])

Loading weights:   0%|          | 0/542 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.1.weight to fine_acoustics.lm_heads.0.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.2.weight to fine_acoustics.lm_heads.1.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.3.weight to fine_acoustics.lm_heads.2.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_l